# **SEGMENTACIJA PLOVILA IZ SATELITSKIH SNIMAKA**

## Autor: Luka Paladin, univ.bacc.ing.el

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from fastai.conv_learner import *
from fastai.dataset import *

import pandas as pd
import numpy as np
import os
from PIL import Image
from sklearn.model_selection import train_test_split
print('done')

> ## Data

In [ ]:
PATH = './'
TRAIN = '../input/airbus-ship-detection/train_v2/'
TEST = '../input/airbus-ship-detection/test_v2/'
SEGMENTATION = '../input/airbus-ship-detection/train_ship_segmentations_v2.csv'
print('done')

In [ ]:
nw = 2           
arch = resnet34  

train_names = [f for f in os.listdir(TRAIN)]
test_names = [f for f in os.listdir(TEST)]
tr_n, val_n = train_test_split(train_names, test_size=0.05, random_state=42)
segmentation_df = pd.read_csv(os.path.join(PATH, SEGMENTATION)).set_index('ImageId')

> ## Funkcije

In [ ]:
def cut_empty(names):
    return [name for name in names 
            if(type(segmentation_df.loc[name]['EncodedPixels']) != float)]

tr_n = cut_empty(tr_n)
val_n = cut_empty(val_n)

def get_mask(img_id, df):
    shape = (768,768)
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    masks = df.loc[img_id]['EncodedPixels']
    if(type(masks) == float): return img.reshape(shape)
    if(type(masks) == str): masks = [masks]
    for mask in masks:
        s = mask.split()
        for i in range(len(s)//2):
            start = int(s[2*i]) - 1
            length = int(s[2*i+1])
            img[start:start+length] = 1
    return img.reshape(shape).T

def get_data(sz,bs):
    #data augmentation
    aug_tfms = [RandomRotate(10, tfm_y=TfmType.CLASS),
                RandomDihedral(tfm_y=TfmType.CLASS),
                RandomLighting(0.05, 0.05, tfm_y=TfmType.CLASS)]
    tfms = tfms_from_model(arch, sz, crop_type=CropType.NO, tfm_y=TfmType.CLASS, 
                aug_tfms=aug_tfms)
    tr_names = tr_n if (len(tr_n)%bs == 0) else tr_n[:-(len(tr_n)%bs)] #cut incomplete batch
    ds = ImageData.get_ds(pdFilesDataset, (tr_names,TRAIN), 
                (val_n,TRAIN), tfms, test=(test_names,TEST))
    md = ImageData(PATH, ds, bs, num_workers=nw, classes=None)
    return md

In [ ]:
class pdFilesDataset(FilesDataset):
    def __init__(self, fnames, path, transform):
        self.segmentation_df = pd.read_csv(SEGMENTATION).set_index('ImageId')
        super().__init__(fnames, transform, path)
    
    def get_x(self, i):
        img = open_image(os.path.join(self.path, self.fnames[i]))
        if self.sz == 768: return img 
        else: return cv2.resize(img, (self.sz, self.sz),cv2.INTER_AREA)
    
    def get_y(self, i):
        mask = np.zeros((768,768), dtype=np.uint8) if (self.path == TEST) \
            else get_mask(self.fnames[i], self.segmentation_df)
        img = Image.fromarray(mask).resize((self.sz, self.sz),cv2.INTER_AREA).convert('RGB')
        return np.array(img).astype(np.float32)
    
    def get_c(self): return 0

> ## Model

In [ ]:
cut,lr_cut = model_meta[arch]
def get_base(pre=True):                   #load ResNet34 model
    layers = cut_model(arch(pre), cut)
    return nn.Sequential(*layers)

In [ ]:
class ResBlock(nn.Module):
    def __init__(self, x_in, filters=64):
        super().__init__()
        self.conv1 = nn.Conv2d(x_in,filters,1)
        self.bn1 = nn.BatchNorm2d(filters)
        self.conv2 = nn.Conv2d(filters,filters,(3,3),padding=1)
        self.bn2 = nn.BatchNorm2d(filters)
        self.conv3 = nn.Conv2d(filters,x_in,1)
        
    def forward(self, x):
        r = self.conv1(x)
        r = F.relu(r)
        r = self.bn1(r)
        r = self.conv2(r)
        r = F.relu(r)
        r = self.bn2(r)
        r = self.conv3(r)
        return x + r
    
class SEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
                nn.Linear(channel, channel // reduction),
                nn.ReLU(inplace=True),
                nn.Linear(channel // reduction, channel),
                nn.Sigmoid())

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y

class UnetBlock(nn.Module):
    def __init__(self, up_in, x_in, n_out, dropout=0.0):
        super().__init__()
        up_out = x_out = n_out//2
        self.x_conv  = nn.Conv2d(x_in,  x_out,  1)
        self.tr_conv = nn.ConvTranspose2d(up_in, up_out, 2, stride=2)
        self.cat_conv = nn.Conv2d(n_out, n_out, (3,3), padding=(1,1))
        self.bn = nn.BatchNorm2d(n_out)
        self.se = SEBlock(n_out)
        self.dropout = nn.Dropout2d(dropout)
        self.r = ResBlock(n_out,n_out//2)
        #self.conv1 = nn.Conv2d(2*n_out,n_out,1)
        #self.conv2 = nn.Conv2d(n_out,n_out,1)
        #self.bn_g = nn.BatchNorm2d(n_out)
        #self.GLU = nn.GLU(dim=1)
        #self.conv1 = nn.Conv2d(n_out,n_out,(3,3), padding=(1,1))
        #self.conv2 = nn.Conv2d(n_out,n_out,(3,3), padding=(1,1))
        #self.bn_in = nn.BatchNorm2d(n_out)
        
    def forward(self, up_p, x_p):
        up_p = self.tr_conv(up_p)
        x_p = self.x_conv(x_p)
        cat_p = self.dropout(torch.cat([up_p,x_p], dim=1))
        x = self.bn(F.relu(self.cat_conv(cat_p)))
        x = self.r(x)
        #g = self.bn_g(self.conv2(F.relu(self.conv1(cat_p))))
        #x = self.GLU(torch.cat([x_p,g], dim=1))
        x = self.se(x)
        return x

class SaveFeatures():
    features=None
    def __init__(self, m): self.hook = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output): self.features = output
    def remove(self): self.hook.remove()
    
class Unet34SE_hc(nn.Module):
    def __init__(self, rn, dropout = 0.0):
        super().__init__()
        self.rn = rn
        self.hc_sz = 16
        self.filters = [64,64,128,256,512]
        self.u_out = [self.hc_sz,128,128,256,256]
        self.sfs = [SaveFeatures(rn[i]) for i in [2,4,5,6]]
        self.up1 = UnetBlock(self.filters[4],self.filters[3],self.u_out[4],dropout)
        self.up2 = UnetBlock(self.u_out[4],self.filters[2],self.u_out[3],dropout)
        self.up3 = UnetBlock(self.u_out[3],self.filters[1],self.u_out[2],dropout/2)
        self.up4 = UnetBlock(self.u_out[2],self.filters[0],self.u_out[1])
        self.up5 = nn.ConvTranspose2d(self.u_out[1], self.u_out[0], 2, stride=2)
        self.hc_neck1 = nn.Conv2d(self.u_out[4], self.hc_sz, 1)
        self.hc_neck2 = nn.Conv2d(self.u_out[3], self.hc_sz, 1)
        self.hc_neck3 = nn.Conv2d(self.u_out[2], self.hc_sz, 1)
        self.hc_neck4 = nn.Conv2d(self.u_out[1], self.hc_sz, 1)
        self.head = nn.Sequential(nn.Conv2d(5*self.hc_sz,2*self.hc_sz,3,padding=1),
                                 nn.ReLU(inplace=True),
                                 nn.Conv2d(2*self.hc_sz,1,1))
        
    def forward(self,x):
        x = F.relu(self.rn(x))
        x = self.up1(x, self.sfs[3].features)
        hc1 = self.hc_neck1(x)
        x = self.up2(x, self.sfs[2].features)
        hc2 = self.hc_neck2(x)
        x = self.up3(x, self.sfs[1].features)
        hc3 = self.hc_neck3(x)
        x = self.up4(x, self.sfs[0].features)
        hc4 = self.hc_neck4(x)
        x = self.up5(x)
        x = torch.cat((x,
            F.interpolate(hc4,scale_factor=2,mode='bilinear'),
            F.interpolate(hc3,scale_factor=4,mode='bilinear'),
            F.interpolate(hc2,scale_factor=8,mode='bilinear'),
            F.interpolate(hc1,scale_factor=16,mode='bilinear'),
            ),dim=1)
        x = self.head(x)
        return x[:,0]
    
    def close(self):
        for sf in self.sfs: sf.remove()
            
class UnetModel():
    def __init__(self,model,name='Unet'):
        self.model,self.name = model,name

    def get_layer_groups(self, precompute):
        lgs = list(split_by_idxs(children(self.model.rn), [lr_cut]))
        return lgs + [children(self.model)[1:]]

> ## Loss funkcije

In [ ]:
def dice_loss(input, target):
    input = torch.sigmoid(input)
    smooth = 1.0

    iflat = input.view(-1)
    tflat = target.view(-1)
    intersection = (iflat * tflat).sum()
    
    return ((2.0 * intersection + smooth) / (iflat.sum() + tflat.sum() + smooth))

class FocalLoss(nn.Module):
    def __init__(self, gamma):
        super().__init__()
        self.gamma = gamma
        
    def forward(self, input, target):
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})"
                             .format(target.size(), input.size()))

        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + \
            ((-max_val).exp() + (-input - max_val).exp()).log()

        invprobs = F.logsigmoid(-input * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        
        return loss.mean()
    
class MixedLoss(nn.Module):
    def __init__(self, alpha, gamma):
        super().__init__()
        self.alpha = alpha
        self.focal = FocalLoss(gamma)
        
    def forward(self, input, target):
        loss = self.alpha*self.focal(input, target) - torch.log(dice_loss(input, target))
        return loss.mean()
    
def dice(pred, targs):
    pred = (pred>0).float()
    return 2.0 * (pred*targs).sum() / ((pred+targs).sum() + 1.0)

def IoU(pred, targs):
    pred = (pred>0).float()
    intersection = (pred*targs).sum()
    return intersection / ((pred+targs).sum() - intersection + 1.0)

> ## Treniranje (256x256)

#### Preuzimanje Unet modela

In [ ]:
m = to_gpu(Unet34SE_hc(get_base(True),0.15))
models = UnetModel(m)
print('done')

#### Definiranje parametara

In [ ]:
sz = 256 #image size
bs = 64  #batch size

md = get_data(sz,bs)

learn = ConvLearner(md, models)
learn.opt_fn=optim.Adamax
learn.clip = 1.0
learn.crit = MixedLoss(10.0, 2.0)
learn.metrics=[accuracy_thresh(0.5),dice,IoU]

wd=1e-7
lr = 2.5e-3

In [ ]:
#learn.freeze_to(1)

In [ ]:
learn.fit(lr,1,wds=wd,cycle_len=1,use_clr=(5,8))
learn.save('Unet34r_256_0')

In [ ]:
lrs = np.array([lr/100,lr/10,lr])
learn.unfreeze() #unfreeze the encoder
learn.bn_freeze(True)

learn.fit(lrs,2,wds=wd,cycle_len=1,use_clr=(20,8))

learn.fit(lrs/3,2,wds=wd,cycle_len=2,use_clr=(20,8))

learn.save('Unet34r_256_1')

## Vizualizacija

In [ ]:
def Show_images(x,yp,yt):
    columns = 3
    rows = min(bs,8)
    fig=plt.figure(figsize=(columns*4, rows*4))
    for i in range(rows):
        fig.add_subplot(rows, columns, 3*i+1)
        plt.axis('off')
        plt.imshow(x[i])
        fig.add_subplot(rows, columns, 3*i+2)
        plt.axis('off')
        plt.imshow(yp[i])
        fig.add_subplot(rows, columns, 3*i+3)
        plt.axis('off')
        plt.imshow(yt[i])
    plt.show()

In [ ]:
learn.model.eval();
x,y = next(iter(md.val_dl))
yp = to_np(F.sigmoid(learn.model(V(x))))

Show_images(np.asarray(md.val_ds.denorm(x)), yp, y)

# Nastavak treniranja (384x384)

In [ ]:
sz = 384 #image size
bs = 16  #batch size

md = get_data(sz,bs)
learn.set_data(md)
learn.unfreeze()
learn.bn_freeze(True)

In [ ]:
learn.fit(lrs/5,3,wds=wd,cycle_len=2,use_clr=(10,8))
learn.save('Unet34r_384_1')

# Vizualizacija 2

In [ ]:
learn.model.eval();
x,y = next(iter(md.val_dl))
yp = to_np(F.sigmoid(learn.model(V(x))))

Show_images(np.asarray(md.val_ds.denorm(x)), yp, y)

# Nastavak treniranja (768x768)

In [ ]:
sz = 768 #image size
bs = 6  #batch size

md = get_data(sz,bs)
learn.set_data(md)
learn.unfreeze()
learn.bn_freeze(True)

In [ ]:
learn.fit(lrs/10,1,wds=wd,cycle_len=1,use_clr=(10,8))
learn.save('Unet34_768_1')

# Vizualizacija 3

In [ ]:
learn.model.eval();
x,y = next(iter(md.val_dl))
yp = to_np(F.sigmoid(learn.model(V(x))))

Show_images(np.asarray(md.val_ds.denorm(x)), yp, y)